# Using FLAML try lightGBM

In [1]:
import pandas as pd
from flaml import AutoML
from sklearn.model_selection import train_test_split


# Load dataset #! Still needs to be adjusted
df = pd.read_csv("your_preprocessed_data.csv")


# Split data into features (X) and target (y)
X = df.drop('posture', axis=1)  # Assuming 'posture' is the target column
y = df['posture']


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# FLAML setup with LightGBM focus
automl = AutoML()

settings = {
    "time_budget": 300,           # Time limit in seconds (adjust as needed)
    "metric": 'accuracy',          # Classification metric (since posture is categorical)
    "task": 'classification',
    "estimator_list": ['lgbm'],   # Only use LightGBM
    "log_file_name": 'flaml_lgbm_posture.log', # Log for detailed output
    "eval_method": "holdout",      # Use holdout validation for time-series
    "early_stop": True,           # Stop early if no improvement is seen
    "n_splits": 5,                # Use 5-fold cross-validation for final model
    "mem_thres": 1024 * 8,         # 8GB memory limit (adjust if needed)
    "n_jobs": -1
}

# Execute FLAML's AutoML process
automl.fit(X_train=X_train, y_train=y_train, **settings)

# Best model and its performance
best_estimator = automl.best_estimator
best_config = automl.best_config
best_loss = automl.best_loss

print(f"Best estimator: {best_estimator}")
print(f"Best hyperparameters: {best_config}")
print(f"Best loss (accuracy): {-best_loss:.4f}") # -best_loss gives accuracy

# Make predictions on new data
# (Important: Make sure this new data is also time-ordered!)
new_X = ...  # Load/preprocess your new accelerometer/gyro data
predictions = automl.predict(X_test)


FileNotFoundError: [Errno 2] File your_preprocessed_data.csv does not exist: 'your_preprocessed_data.csv'

# Now plots to get visual representation of the training

## Learning Curves

In [ ]:
import matplotlib.pyplot as plt
from flaml.data import get_output_from_log
import seaborn as sns

# Load FLAML's log file (replace with your actual log file name)
df = get_output_from_log(log_file_name='flaml_lgbm_posture.log', time_budget=300)

# 1. Learning Curves:
plt.figure(figsize=(12, 6))
for config_id, config_data in df.groupby('config_id'):
    plt.plot(config_data['wall_clock_time'], config_data['val_loss'], label=f'Config {config_id}')
plt.xlabel('Time (s)')
plt.ylabel('Validation Loss (Accuracy)')
plt.title('Learning Curves for Different Configurations')
plt.legend()
plt.show()




## Hyperparameter Importance

In [ ]:
# 2. Hyperparameter Importance:
importances = automl.model.estimator.feature_importances_
features = X.columns
plt.figure(figsize=(10, 6))
sns.barplot(x=features, y=importances)
plt.xticks(rotation=45)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.show()


## Confusion Matrix

In [ ]:
# 3. Confusion Matrix:
from sklearn.metrics import confusion_matrix
y_pred = automl.predict(X)
cm = confusion_matrix(y, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=automl.classes_, yticklabels=automl.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

##  Precision-Recall Curve (for each posture class)

In [ ]:
# 4. Precision-Recall Curve (for each posture class):
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize

y_bin = label_binarize(y, classes=automl.classes_)
n_classes = y_bin.shape[1]

plt.figure(figsize=(10, 8))
for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(y_bin[:, i], automl.predict_proba(X)[:, i])
    plt.plot(recall, precision, label=f'Class {automl.classes_[i]}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

## Classification Report (summary of metrics)

In [ ]:

# 5. Classification Report (summary of metrics):
from sklearn.metrics import classification_report
print(classification_report(y, y_pred, target_names=automl.classes_))